# Tutorial 6 - Sieci neuronowe

## 1) Sieci MLP - multi-layer perceptron
### 1.1) Zasada działania

MLP (perceptron wielowarstwowy) jest algorytmem _uczenia nadzorowanego_. Dla danego zbioru wektorów cech X oraz wektora odpowiedzi y trenuje on _nieliniową_ fukcję $f: R^m \rightarrow R^o$, gdzie $m$ jest liczbą wymiarów wektorów wejściowych a $o$ liczbą wymiarów wektorów wyjściowych. Może być ona używana zarówno do rozwiązywania problemów klasyfikacji, jak i regresji.

Trenowana funkcja $f$ ma postać kilkuwarstwowej sieci. Każda warstwa złożona jest z pewnej ilości neuronów. Pierwsza warstwa jest _warstwą wejściową_, złożoną z $m$ neuronów. Kolejne warstwy złożone są z pewnej ilości neuronów, których wartości zależą od wartości w poprzedniej warstwie. Ostatnia warstwa - _warstwa wyjściowa_ - składa się z $o$ neuronów. Warstwy pomiędzy warstwą wejściową i wyjściową nazywamy _warstwami ukrytymi_. Wartość funkcji $f$ dla wektora $x \in R^m$ jest wartość neuronów wyjściowych dla wartości neuronów wejściowych równej $x$.

<img src="images/multilayerperceptron_network.png" style="width: 350px; text-align: left" />
Rysunek 1: sieć multi-layer perceptron (źródło: https://scikit-learn.org)

Wartość neuronu w warstwie wejściowej zależy od danych wejściowych. Wartość neuronu w warstwach ukrytych i warstwie wyjściowej zależy od sumy ważonej (kombinacji liniowej) wartości neuronów i dodatkowej wartości - bias'u. Dodatkowo, najlepiej aby wartość neuronu $\in [0, 1]$. W tym celu:
* dla _warstwy wejściowej_ - normalizujemy dane
* dla _warstw ukrytych_ i _warstwy wyjściowej_ - obliczoną sumę przepuszczamy przez __funkcję aktywacji__, której dziedziną jest $R$ a zbiorem wartości jest $[0, 1]$ . Przykładową funkcją aktywacji jest sigmoid: $f(x) = (1 + e^{-x})^{-1}$ (wykres 1)

In [ ]:
import math

import matplotlib.pyplot as plt
import numpy as np


def sigmoid(x):
    return 1 / (1 + math.exp(-x))

x = np.linspace(-10, 10, 100)
y = [sigmoid(i) for i in x]
plt.plot(x, y)
plt.show()

Wykres 1: Funkcja sigmoid

Możemy zauważyć, że obliczenie wartości neuronów dla warstwy składa się z dwóch kroków:
1. _przekształcenie affiniczne_ wektora wartości neuronów z poprzedniej warstwy. Przekształcenie affiniczne jest to przekształcenie w postaci $x \rightarrow f(x) + b$, gdzie f jest przekształceniem liniowym a b wektorem przesunięcia.
2. obliczenie wartości poszczególnych neuronów na podstawie wartości wektora uzyskanych w punkcie (1) z użyciem funkcji aktywacji

Przekształcenie affiniczne wektora $X$ w $X'$ możemy zamienić w przekształcenie liniowe, dodając do wektora X dodatkową współrzędną o wartości 1. Umożliwia to dodanie stałego współczynnika do kombinacji liniowych - mnożąc wagę razy 1 otrzymamy zawsze daną wagę (patrz rysunek 1). Dzięki temu, możemy przedstawić współczynniki w postaci macierzy przekształcenia liniowego, a samo przekształcenie zrealizować jako mnożenie macierzy przez wektor.

### 1.2) Trenowanie sieci
Parametrami funkcji f, które trenujemy, są:
* wagi średniej ważonej (`coefs_`) - dla każdego neuronu w warstwie $l$ wag jest tyle, ile neuronów w warstwie $l - 1$,
* bias'y (`intercepts_`) - po jednym dla każdego neuronu.

Jeśli ilość neuronów w warstwie l oznaczymy jako $n_l$, dla każdej warstwy mamy $(n_{l - 1} + 1) n_l$ wartości do wyznaczenia. Jeśli mamy L warstw w naszej sieci MLP, mamy $\sum_{l=2}^{L} ((n_{l - 1} + 1) n_l)$ wartości do wyznaczenia.

Trenowanie sieci neuronowej polega na minimalizacji funkcji strat (loss function) na danym zbiorze treningowym, wyrażającej "koszt" sieci neuronowej. Standardowo używa się sumy błędów średniokwadratowych pomiędzy wynikami sieci neuronowej a wartościami oczekiwanymi dla kolejnych wektorów wejściowych. Parametrami funkcji loss są wszystkie parametry (wagi średnich ważonych oraz bias'y) naszej sieci neuronowej. Ilość tych parametrów jest bardzo duża, dlatego nie stosuje się standardowej metody wyznaczania ekstremów funkcji wielu zmiennych (przyrównywanie pochodnych cząstkowych do zera). Zamiast tego, istnieją specjalne metody:
* `sgd` - schotastic gradient descent
* `adam` - another schotastic gradient descent
* `lbfgs` - another optimizer

TODO: more about this

Na kanale 3blue3brown w serwisie YouTube znajduje się 4-odcinkowa seria ilustrująca działanie sieci neuronowej MLP z użyciem schotastic gradient descent na zbiorze MNIST - https://www.youtube.com/playlist?list=PLZHQObOWTQDNU6R1_67000Dx_ZCJB-3pi.

### 1.3) Przykład
Na potrzeby tego tutoriala, wykorzystamy bibliotekę `scikit-learn`. Wykorzystuje ona jedynie CPU, przez co nie nadaje się do zastosowań o dużej skali. Obliczenia związane z sieciami neuronowymi dziś wykorzystuje się zazwyczaj z użyciem GPU lub specjalizowanych układów stworzonych do tego celu (np. opartych na FPGA lub będących częściami nowoczesnych układów SoC w smartfonach). Biblioteki takie jak `tensorflow`, `keras` bądź `pytorch` umożliwiają działania na sieciach neuronowych z wykorzystaniem GPU.

Działanie sieci MLP pokażemy standardowo na zbiorze `MNIST`:

In [ ]:
import tensorflow as tf # korzystamy jedynie w celu pobrania zbioru MNIST

(X_train, y_train), (X_test, y_test) = tf.keras.datasets.mnist.load_data()

X_train = X_train.reshape((X_train.shape[0], X_train.shape[1] * X_train.shape[2]))
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1] * X_test.shape[2]))

Biblioteka `scikit-learn` udostępnia klasę `MLPClassifier`, implementującą algorytm MLP używający backpropagation do trenowania.

Konstruktor klasy `MLPClassifier` pozwala na ustawienie wielu parametrów, ich pełna lista dostępna jest w dokumentacji `scikit-learn`. Najważniejsze z nich to:
* __solver__ - metoda użyta do trenowania sieci. Dostępne metody: `"sgd"`\\`"adam"`\\`"lbfgs"`
* __hidden_layer_sizes__ -  lista rozmiarów warstw ukrytych
* __activation__ - użyta funkcja aktywacji. Dostępne funkcje aktywacji:
    * `'identity'` - funkcja tożsamościowa
    * `'logistic'` - sigmoid
    * `'tanh'` - tangens hiperboliczny
    * `'relu'` - $f(x) = max(0, x)$
* __alpha__ - wartość parametru regularyzacji - pozwala na ograniczanie rozmiaru wag. Odpowiednie dobranie tego parametru pozwala na ograniczenie overfittingu/underfittingu.
* __random_state__ - seed funkcji losowej, użytej do generowania wag. Aby wszystkie sieci miały początkowo takie same wagi, wartość parametru random_state musi być jednakowa.
* __max_iter__ - maksymalna ilość iteracji trenowania

In [ ]:
SEED = 6

Klasy `MLPClassifier` używamy podobnie, jak innych klasyfikatorów będących częścią biblioteki sklearn.

In [ ]:
from sklearn.neural_network import MLPClassifier

clf = MLPClassifier(
    solver='sgd', 
    hidden_layer_sizes=(50,), 
    random_state=SEED, 
    alpha=1e-5, 
    verbose=10,
    max_iter=100
)
clf.fit(X_train, y_train)

Używając zbioru testowego MNIST sprawdźmy, ile wynosi accuracy score naszego modelu.

In [ ]:
from sklearn.metrics import accuracy_score

y_pred = clf.predict(X_test)
y_test
accuracy_score(y_test, y_pred)

Obiekty klasy `MLPClassifier` udostępniają m. in. następujące atrybuty:
* `loss_` - obecna wartość funkcji loss
* `coefs_` - lista macierzy wag dla poszczególnych warstw
* `intercepts_` - lista wektorów bias dla poszczególnych warstw

In [ ]:
clf.loss_

Wyświetlmy wartości współczynników macierzy wag dla niektórych neuronów w warstwie ukrytej w postacji obrazków:

In [ ]:
for i in range(10):
    coefs = clf.coefs_[0][:,i]
    coefs = coefs.reshape(28, 28)
    plt.imshow(coefs, cmap='Greys')
    plt.show()

Możemy zauważyć, że dla _większości_ obrazków wagi macierzy aktywacji nie reprezentują konkretnych kształtów.

Wyświetlmy dodatkowo wartości bias dla tych neuronów:

In [ ]:
clf.intercepts_[0][:10]

### 1.4) Zadanie

Pokazać jak zmiany parametrów sieci (wielkość sieci dobrać do wielkości danych i możliwości obliczeniowych) tj.
optymalizator, regularyzacja (np., drop-out), funkcja aktywacji i inne wpływa na wynik.
Ważne: __sieć ma być trenowana dla identycznych wag początkowych__ (należy użyć sieci o takim samym rozmiarze i zanicjować sieć z użyciem takiego samego seed'a (parametr `random_state`). 
W sprawozdaniu należy umieścić tabelkę zawierającą rezultaty i wnioski.

| solver | hidden_layers_sizes | activation | alpha | (other params...) | loss | accuracy_score |
|--------|---------------------|------------|-------|-------------------|------|----------------|
|        |                     |            |       |                   |      |                |
|        |                     |            |       |                   |      |                |
|        |                     |            |       |                   |      |                |

In [ ]:
# rozwiązanie zadania

# 2) Wpływ długości uczenia sieci na błąd